In [1]:
# import numpy as np
# import pandas as pd
# import heartpy as hp
# from pyhrv import time_domain
# from pyhrv import hrv
# import pyhrv

# %matplotlib agg


# import os
# os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export')
# txt_file_path= 'candidate1_HRV_2023-05-18_rowing_3.0.csv'

# # Load your data from the txt file
# # data = pd.read_csv('p1.txt', delimiter='\t', header=None, names=['time_elapsed', 'ibi'])
# txt_data = pd.read_csv(txt_file_path, sep=',', header='infer')  

# # with open('p1.txt') as f:
# #     lines = f.readlines()
# #     time_elapsed = np.array([line.split()[0] for line in lines])
# #     ibi = np.array([line.split()[1] for line in lines])
# # time_elapsed = time_elapsed.astype(np.float64)
# # ibi = ibi.astype(np.float64)


# # data_dict = {'time_elapsed': time_elapsed[2395:], 'ibi': ibi[2395:]}
# # data = pd.DataFrame(data_dict)
# # print(data['time_elapsed'])
# # data['time_elapsed'] = np.array(data['time_elapsed']/60)

# txt_data['timestamp'] = np.array(txt_data['timestamp']/60)

# # Assuming your time_elapsed is in seconds
# fs = 1 / np.mean(np.diff(txt_data['timestamp']))


# # nni = pyhrv.utils.load_sample_nni()

# # Function to compute HRV features on a 5-minute moving window
# def compute_hrv_features(ibis):
#     hrv_sample = time_domain.nni_parameters(ibis)
#     return hrv_sample['nni_mean']

# # Initialize variables
# window_size = int(fs) * 5  # 5 minutes in minuttes
# num_samples = len(txt_data['hrv'])
# hrv_samples = []

# # Iterate through the IBI array with a 5-minute moving window
# for i in range(num_samples - window_size + 1):
#     window = txt_data.iloc[i:i+window_size]
#     window_data=window['hrv']
#     window_time = window['timestamp']

#     # print(window_time)
#     # print(window_data.values)
#     hrv_sample = compute_hrv_features(window_data)
#     hrv_samples.append(hrv_sample)

# # Display or save the HRV samples as needed
# for i, hrv_sample in enumerate(hrv_samples):
#     print(f'HRV Sample {i+1}: {hrv_sample}')


# print('Mean all HRV ', np.mean(hrv_samples))


In [2]:
# import numpy as np
# import pandas as pd
# import heartpy as hp
# import pyhrv.time_domain as td
# from pyhrv import hrv
# import pyhrv
# import glob

# import os


# os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/test')

# files = glob.glob('*.csv')  # Adjust the path as needed

# # Function to compute HRV features on a 5-minute moving window
# def compute_hrv_features(ibis):
#     hrv_sample = td.time_domain(ibis)
#     return hrv_sample['nni_mean']


# result=pd.DataFrame({'mean_RR':[], 'file': [], 'candidate': [], 'feeling': []})

# for file in files: 
#     print(file)
#     txt_data = pd.read_csv(file, sep=',', header='infer')  

#     txt_data['timestamp'] = np.array(txt_data['timestamp']/60)

#     # Assuming your time_elapsed is in seconds
#     fs = 1 / np.mean(np.diff(txt_data['timestamp']))

#     # Initialize variables
#     window_size = int(fs) * 5  # 5 minutes in minuttes
#     num_samples = len(txt_data['hrv'])
#     hrv_samples = []
#     temp_result=pd.DataFrame()

#     # Iterate through the IBI array with a 5-minute moving window
#     for i in range(num_samples - window_size + 1):
#         window = txt_data.iloc[i:i+window_size]
#         window_data=window['hrv']
#         window_time = window['timestamp']

#         # print(window_time)
#         # print(window_data.values)
#         hrv_sample = compute_hrv_features(window_data)
#         hrv_samples.append(hrv_sample)

#     temp_result['mean_RR'] = hrv_samples
#     temp_result['file'] = file
#     temp_result['candidate'] = 1
#     temp_result['feeling'] = txt_data['feeling'][0]
#     result = result.append(temp_result)

#     print(result.shape)
#     # Display or save the HRV samples as needed
#     # for i, hrv_sample in enumerate(hrv_samples):
#     #     print(f'HRV Sample {i+1}: {hrv_sample}')


# print('Mean all HRV ', np.mean(hrv_samples))
# print(result)

In [3]:
import numpy as np
from pyhrv import time_domain, frequency_domain, nonlinear
import hfda
import tsfel
from scipy.stats import skew 
from scipy.stats import kurtosis 
from hrvanalysis import get_frequency_domain_features, get_sampen
import pandas as pd

%matplotlib agg



def calculate_hrv_features_pyhrv(rr_intervals):
    # Convert RR intervals to milliseconds
    rr_intervals_ms = np.array(rr_intervals)

    # Time domain features
    time_domain_features = time_domain.time_domain(rr_intervals_ms, threshold=25)
    sdsd_feature = time_domain.sdsd(rr_intervals)
    kurt_feature = kurtosis(rr_intervals_ms)
    skew_feature= skew(rr_intervals_ms)


    # Frequency domain features
    frequency_domain_features_anlysis = get_frequency_domain_features(rr_intervals_ms)

    # Nonlinear features
    nonlinear_features = nonlinear.poincare(rr_intervals_ms)

    # Additional features
    sampen_value= get_sampen(rr_intervals_ms)

    # higuci_value = hfda.measure(rr_intervals, 5)

    # Combine all features into a dictionary
    hrv_features = {
        'MEAN_RR': time_domain_features['nni_mean'],
        'MEDIAN_RR': np.median(rr_intervals),
        'SDRR': time_domain_features['sdnn'],
        'RMSS': time_domain_features['rmssd'],
        'SDSD': sdsd_feature['sdsd'],
        'SDRR_RMSSD': time_domain_features['sdnn'] / time_domain_features['rmssd'],
        'HR': 60000 / time_domain_features['nni_mean'],
        'pNN25': time_domain_features['pnn25'],
        'pNN50': time_domain_features['pnn50'],
        'SD1': nonlinear.poincare(rr_intervals_ms)['sd1'],
        'SD2': nonlinear.poincare(rr_intervals_ms)['sd2'],
        'KURT': kurt_feature,
        'SKEW': skew_feature,
        'VLF': frequency_domain_features_anlysis['vlf'],
        'VLF_PCT': frequency_domain_features_anlysis['vlf']*100 / frequency_domain_features_anlysis['total_power'],  
        'LF': frequency_domain_features_anlysis['lf'],
        'LF_PCT': frequency_domain_features_anlysis['lf']*100 / frequency_domain_features_anlysis['total_power'],
        'LF_NU': frequency_domain_features_anlysis['lfnu'],
        'HF': frequency_domain_features_anlysis['hf'], 
        'HF_PCT': frequency_domain_features_anlysis['hf']*100 / frequency_domain_features_anlysis['total_power'],
        'HF_NU': frequency_domain_features_anlysis['hfnu'],
        'TP': frequency_domain_features_anlysis['total_power'], 
        'LF_HF': frequency_domain_features_anlysis['lf_hf_ratio'],
        'HF_LF': 1/frequency_domain_features_anlysis['lf_hf_ratio'],
        'sampen': sampen_value['sampen']
    }
    return hrv_features



# Example usage
# rr_intervals_example = [1000, 1050, 1100, 1000, 950, 1050, 1100, 1000, 950, 1050]
# hrv_result = calculate_hrv_features_pyhrv(rr_intervals_example)
# print(hrv_result)


In [4]:
import os
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/convertion')
import glob
import warnings
warnings.filterwarnings('ignore')

files = glob.glob('*.csv')  # Adjust the path as needed

# # Function to compute HRV features on a 5-minute moving window
# def compute_hrv_features(ibis):
#     hrv_sample = td.time_domain(ibis)
#     return hrv_sample['nni_mean']


# result=pd.DataFrame({'mean_RR':[], 'file': [], 'candidate': [], 'feeling': []})
file_number = 1

for file in files:     
    file_name = str(file)
    print(f'{file_number} of {len(files)}')
    print(f'Begin converting {file_name}')
    output_file = file_name[:-4] + '_features.txt'

    txt_data = pd.read_csv(file, sep=',', header='infer')  
    txt_data = txt_data[240:len(txt_data)-240]
    txt_data['timestamp'] = np.array(txt_data['timestamp'])
    txt_feeling = np.array(txt_data['feeling'])

    print('Length of file: ', len(txt_data))

    # Assuming your time_elapsed is in seconds
    fs = 1 / np.mean(np.diff(txt_data['timestamp']))


    # Initialize variables
    window_size = int(fs) * 60 * 5  # 5 minutes in minuttes
    num_samples = len(txt_data['hrv'])
    hrv_samples = []
    temp_result=pd.DataFrame()


    print('Expected convertion lenght: ', int((len(txt_data)-window_size+1)/(fs*10)))


    total = calculate_hrv_features_pyhrv(txt_data['hrv'])
    keys = list(total.keys())
    header_string = ', '.join(map(str, keys))
    header_string_all = 'File, ' + header_string + ', Feeling'
    with open(output_file, 'a') as file1:
            file1.write(header_string_all + '\n')  # Add a newline after each set of values


    # Iterate through the IBI array with a 5-minute moving window
    for i in range(0, num_samples - window_size + 1, int(fs*10)):
        window = txt_data.iloc[i:i+window_size]
        window_data=window['hrv']*1000

        hrv_sample = calculate_hrv_features_pyhrv(window_data)

        # Extract values from the dictionary
        values = list(hrv_sample.values())

        # Convert values to a string
        values_string = ', '.join(map(str, values))
        values_string_all = file_name[:-4] + ', ' + values_string + ', ' + str(txt_feeling[0])

        # Open the output file in append mode and write the values string to it
        with open(output_file, 'a') as file1:
            file1.write(values_string_all + '\n')  # Add a newline after each set of values

    file_number += 1
    print(f'Complete {file_name}')
    print('')
            

print("----------------------")
print('SUCCESS! All files in folder converted')
print("----------------------")

1 of 13
Begin converting candidate2_HRV_2023-04-28_cycling_7.0.csv
Length of file:  12307
Expected convertion lenght:  614
Complete candidate2_HRV_2023-04-28_cycling_7.0.csv

2 of 13
Begin converting candidate2_HRV_2023-05-28_cycling_7.0.csv
Length of file:  14035
Expected convertion lenght:  606
Complete candidate2_HRV_2023-05-28_cycling_7.0.csv

3 of 13
Begin converting candidate2_HRV_2023-06-20_cycling_7.0.csv
Length of file:  11348
Expected convertion lenght:  569
Complete candidate2_HRV_2023-06-20_cycling_7.0.csv

4 of 13
Begin converting candidate2_HRV_2023-06-24_cycling_7.0.csv
Length of file:  11525
Expected convertion lenght:  582
Complete candidate2_HRV_2023-06-24_cycling_7.0.csv

5 of 13
Begin converting candidate2_HRV_2023-07-20_cycling_7.0.csv
Length of file:  10150
Expected convertion lenght:  415
Complete candidate2_HRV_2023-07-20_cycling_7.0.csv

6 of 13
Begin converting candidate2_HRV_2023-07-27_running_7.0.csv
Length of file:  6134
Expected convertion lenght:  251


KeyboardInterrupt: 